In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

from dumbrain.ml.kaggle.download import download
from dumbrain.lib.cleaners import *

%matplotlib inline

In [4]:
data_files = download( 'titanic' )
test_file, train_file, example_output = list( map( lambda file: 'data/' + file, data_files ) )

In [5]:
all_train_data = pd.read_csv( train_file )
all_test_data = pd.read_csv( test_file )
all_example_data = pd.read_csv( example_output )

In [7]:
train_data_uncleaned = all_train_data.sample( frac=0.8, random_state=4111 )
validate_data_uncleaned = all_train_data.drop( train_data_uncleaned.index )

In [9]:
cleaners = [
    RemoveColumnCleaner( 'PassengerId' ),
#     NameSentimentColumnCleaner( 'Name', train_data_uncleaned, 'Survived', omit=[ 'mrs', 'miss', 'mr' ] ),
    RemoveColumnCleaner( 'Name' ),  
    DummyColumnCleaner( 'Sex', [ 'male', 'female' ] ),
    RemoveColumnCleaner( 'Sex' ),
    DummyColumnCleaner( 'Pclass', [ 1, 2, 3 ] ),
    RemoveColumnCleaner( 'Ticket' ),                     # Todo: Use this data
    RemoveColumnCleaner( 'Cabin' ),                      # Todo: Use this data
    DummyColumnCleaner( 'Embarked', [ 'S', 'C', 'Q' ] ),
    FillNaNDataCleaner( 0 )
]

train_data = cleanData( cleaners, train_data_uncleaned )
validate_data = cleanData( cleaners, validate_data_uncleaned )

In [10]:
model = RandomForestClassifier()

In [11]:
train_data.columns

Index(['Survived', 'Age', 'SibSp', 'Parch', 'Fare', '_female_Sex', '_male_Sex',
       '_1_Pclass', '_2_Pclass', '_3_Pclass', '_C_Embarked', '_Q_Embarked',
       '_S_Embarked'],
      dtype='object')

In [18]:
model.fit( train_data.drop( 'Survived', axis=1 ), train_data[ 'Survived' ] )

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [19]:
model.score( validate_data.drop( 'Survived', axis=1 ), validate_data[ 'Survived' ] )

0.8033707865168539

In [20]:
test_data_cleaned = cleanData( cleaners, all_test_data )
predicted = model.predict( test_data_cleaned )

output = pd.DataFrame( { 'PassengerId': all_test_data[ 'PassengerId' ], 'Survived': predicted } )
output = output.set_index( 'PassengerId' )

In [21]:
output.to_csv( 'data/results.csv' )